# <p class="lead"><i>This Jupyter notebook performs Wrangling and combining of Data for six pollutants Ozone, SO2, NO2, CO, PM2.5 and PM 10 </i></p>

In [25]:
import pandas as pd
import math
import os
from pathlib import Path
import glob

In [18]:
ftype = ['daily_44201_*.csv','daily_42401_*.csv',"daily_42602_*.csv","daily_42101_*.csv",'daily_88101_*.csv','daily_81102_*.csv']
finalname = ['Summarized_daily_ozone.csv','Summarized_daily_SO2.csv', 'Summarized_daily_NO2.csv','Summarized_daily_CO.csv','Summarized_daily_PM2.5.csv','Summarized_daily_PM10.csv']

def merge(allFiles,fileName):
    filepath = Path(fileName)
    filelist = glob.glob(allFiles)  
    heading = 0
    if(filelist) != 0:
        print("Total",len(filelist), allFiles ,"exists")
        with open(fileName, 'w',encoding='utf-8', newline="") as file:
            for f in filelist:                                                                                                                    
                print("Processing",f)
                df = pd.read_csv(f, skipinitialspace=True)
                if heading == 0:
                    df.columns = ['State_Code','County_Code','Site_Num','Parameter_Code','POC','Latitude','Longitude','Datum','Parameter_Name','Sample_Duration','Pollutant_Standard','Date_Local','Units_of_Measure','Event_Type' ,'Observation_Count','Observation_Percent','Arithmetic_Mean','Max_Value','1st_Max_Hour','AQI','Method_Code','Method_Name','Local_Site_Name','Address','State_Name','County_Name','City_Name','CBSA_Name','Date_of_Last_Change']
                    df.to_csv(file, header=True,index=False)
                    heading = 1 
                else:
                    df.to_csv(file, header=False, index=False, mode='a')
            print(fileName,"generated!")
            print("----------------------------------------")
    else:
        print("file list is empty!!")
        
for i in range(len(ftype)):
    merge(ftype[i],finalname[i])


Total 6 daily_44201_*.csv exists
Processing daily_44201_2012.csv
Processing daily_44201_2013.csv
Processing daily_44201_2014.csv
Processing daily_44201_2015.csv
Processing daily_44201_2016.csv
Processing daily_44201_2017.csv
Summarized_daily_ozone.csv generated!
----------------------------------------
Total 6 daily_42401_*.csv exists
Processing daily_42401_2012.csv
Processing daily_42401_2013.csv
Processing daily_42401_2014.csv
Processing daily_42401_2015.csv
Processing daily_42401_2016.csv
Processing daily_42401_2017.csv
Summarized_daily_SO2.csv generated!
----------------------------------------
Total 6 daily_42602_*.csv exists
Processing daily_42602_2012.csv
Processing daily_42602_2013.csv
Processing daily_42602_2014.csv
Processing daily_42602_2015.csv
Processing daily_42602_2016.csv
Processing daily_42602_2017.csv
Summarized_daily_NO2.csv generated!
----------------------------------------
Total 6 daily_42101_*.csv exists
Processing daily_42101_2012.csv
Processing daily_42101_2013

In [50]:

def merge_3_col(df1):
    df1['Site_Num'] = df1['Site_Num'].astype('str').str.zfill(4)
    df1['County_Code'] = df1['County_Code'].astype('str').str.zfill(3)
    df1['State_Code'] = df1.State_Code.astype('str').str.cat(df1.County_Code).str.cat(df1.Site_Num)
    df1.drop(df1.columns[[1, 2]], axis=1,inplace=True)
    df1[['Date_Local']] = df1[['Date_Local']].astype('str')
    df1['Date_Local'] = df1['Date_Local'].str.replace('-','')
    df1[['Date_Local']] = df1[['Date_Local']].astype('int64')
    return df1
    
def drange(x,start, stop):
    print("in drange", x)
    if (x >= start) & (x <= stop):
        r = 1
    else:
        r = 0
    return r  

def rewrite_file(df, filename):
    with open(filename, 'w',encoding='utf-8', newline="") as file:
        df.to_csv(file, header=True,index=False)

In [34]:
#os.chdir('SO2daily')
df = pd.read_csv('Summarized_daily_SO2.csv',low_memory=False)
print(df.isnull().sum())
df.drop(['Method_Code','CBSA_Name','Parameter_Code','Parameter_Name','Units_of_Measure','Sample_Duration','Pollutant_Standard','Event_Type','Local_Site_Name','Address','State_Name','County_Name','City_Name','CBSA_Name','Date_of_Last_Change','Method_Code','Method_Name'],axis=1,inplace =True)
df.drop(['Sample_Duration'] =='3-HR BLK AVG',inplace =True)
df = merge_3_col(df)
df = cleanSO2(df)
rewrite_file(df, 'Summarized_daily_SO2.csv')



State_Code                  0
County_Code                 0
Site_Num                    0
Parameter_Code              0
POC                         0
Latitude                    0
Longitude                   0
Datum                       0
Parameter_Name              0
Sample_Duration             0
Pollutant_Standard          0
Date_Local                  0
Units_of_Measure            0
Event_Type                  0
Observation_Count           0
Observation_Percent         0
Arithmetic_Mean             0
Max_Value                   0
1st_Max_Hour                0
AQI                    845561
Method_Code            845561
Method_Name                 0
Local_Site_Name        193135
Address                   213
State_Name                  0
County_Name                 0
City_Name                   0
CBSA_Name              152900
Date_of_Last_Change         0
dtype: int64
in drange 0.5
in 1st range 0.5
Cleaned 1 rows


In [31]:
def cleanSO2(df):
    df['AQI'] = df['AQI'].fillna(5000) 
    count = 0 
    for i, row in df.iterrows():
        if row['Max_Value'] <= 0:
            df = df.set_value(i, 'Max_Value' ,1)
        elif row['Max_Value'] >= 1004:
            df = df.set_value(i, 'Max_Value' ,1003)
        if row['AQI'] == 5000:  
            count =  count + 1
            c = row['Max_Value']
            if  drange(c,0,35.99) == 1:
                #print("in 1st range", c)
                aqi = (50/35)*c
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
            elif drange(c,36,75.99) == 1:
                #print("in 2nd range", c)
                aqi = ((49/39)*(c-36)) + 51
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
                #print("in 2nd range aqi", df4['AQI'])
            elif drange(c,76,185.99)== 1:
                aqi = ((49/109)*(c-76)) + 101
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
                #print("in 3rd range aqi", row['AQI'])
            elif drange(c,186,304.99)== 1:
                aqi = ((49/118)*(c-186)) + 151
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
            elif drange(c,305,604.99)== 1:
                aqi = ((99/299)*(c-305)) + 201
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
            elif drange(c,605,804.99)== 1:
                aqi = ((99/199)*(c-605)) + 301
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
            elif drange(c,805,1004)== 1:
                aqi = ((99/199)*(c-805)) + 401
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
    print('Cleaned',count, "rows")
    return df


In [69]:
df = pd.read_csv('Summarized_daily_ozone.csv',low_memory=False)
print(df.isnull().sum())
df.drop(['Method_Code','CBSA_Name','Parameter_Code','Parameter_Name','Units_of_Measure','Sample_Duration','Pollutant_Standard','Event_Type','Local_Site_Name','Address','State_Name','County_Name','City_Name','CBSA_Name','Date_of_Last_Change','Method_Code','Method_Name'],axis=1,inplace =True)
df = merge_3_col(df)
rewrite_file(df, 'Summarized_daily_ozone.csv')
print(df.isnull().any().any())

State_Code                   0
County_Code                  0
Site_Num                     0
Parameter_Code               0
POC                          0
Latitude                     0
Longitude                    0
Datum                        0
Parameter_Name               0
Sample_Duration              0
Pollutant_Standard           0
Date_Local                   0
Units_of_Measure             0
Event_Type                   0
Observation_Count            0
Observation_Percent          0
Arithmetic_Mean              0
Max_Value                    0
1st_Max_Hour                 0
AQI                          0
Method_Code            2001463
Method_Name                  0
Local_Site_Name         104627
Address                      0
State_Name                   0
County_Name                  0
City_Name                    0
CBSA_Name               207187
Date_of_Last_Change          0
dtype: int64
False


In [50]:
df = pd.read_csv('Summarized_daily_NO2.csv',low_memory=False)
print(df.isnull().sum())
df.drop(['Method_Code','CBSA_Name','Parameter_Code','Parameter_Name','Units_of_Measure','Sample_Duration','Pollutant_Standard','Event_Type','Local_Site_Name','Address','State_Name','County_Name','City_Name','CBSA_Name','Date_of_Last_Change','Method_Code','Method_Name'],axis=1,inplace =True)
df = merge_3_col(df)
rewrite_file(df, 'Summarized_daily_NO2.csv')
print(df.isnull().any().any())

State_Code                 0
County_Code                0
Site_Num                   0
Parameter_Code             0
POC                        0
Latitude                   0
Longitude                  0
Datum                      0
Parameter_Name             0
Sample_Duration            0
Pollutant_Standard         0
Date_Local                 0
Units_of_Measure           0
Event_Type                 0
Observation_Count          0
Observation_Percent        0
Arithmetic_Mean            0
Max_Value                  0
1st_Max_Hour               0
AQI                        0
Method_Code                0
Method_Name                0
Local_Site_Name        52434
Address                    0
State_Name                 0
County_Name                0
City_Name                  0
CBSA_Name              63788
Date_of_Last_Change        0
dtype: int64
False


In [65]:
df = pd.read_csv('Summarized_daily_CO.csv',low_memory=False)
print(df.isnull().sum())
df.drop(['Method_Code','CBSA_Name','Parameter_Code','Parameter_Name','Units_of_Measure','Sample_Duration','Pollutant_Standard','Event_Type','Local_Site_Name','Address','State_Name','County_Name','City_Name','CBSA_Name','Date_of_Last_Change','Method_Code','Method_Name'],axis=1,inplace =True)
df = merge_3_col(df)
df = cleanCO(df)
rewrite_file(df, 'Summarized_daily_CO.csv')
print(df.isnull().any().any())

State_Code                  0
County_Code                 0
Site_Num                    0
Parameter_Code              0
POC                         0
Latitude                    0
Longitude                   0
Datum                       0
Parameter_Name              0
Sample_Duration             0
Pollutant_Standard          0
Date_Local                  0
Units_of_Measure            0
Event_Type                  0
Observation_Count           0
Observation_Percent         0
Arithmetic_Mean             0
Max_Value                   0
1st_Max_Hour                0
AQI                    558069
Method_Code            558112
Method_Name                 0
Local_Site_Name         80577
Address                     0
State_Name                  0
County_Name                 0
City_Name                   0
CBSA_Name               22260
Date_of_Last_Change         0
dtype: int64
Cleaned 0 rows
False


In [53]:
def cleanCO(df):
    df['AQI'] = df['AQI'].fillna(5000) 
    count = 0 
    for i, row in df.iterrows():
        if row['Max_Value'] < 0:
            df = df.set_value(i, 'Max_Value' ,0)
        elif row['Max_Value'] > 50.4:
            df = df.set_value(i, 'Max_Value' ,50.4)
    if row['AQI'] == 5000:  
        count =  count + 1
        c = row['Max_Value']
        if  drange(c,0,4.4) == 1:
            #print("in 1st range", c)
            aqi = (50/4.4*c)
            df = df.set_value(i, 'AQI' ,math.floor(aqi))
        elif drange(c,4.5,9.4) == 1:
            #print("in 2nd range", c)
            aqi = ((49/4.9)*(c-4.5)) + 51
            df = df.set_value(i, 'AQI' ,math.floor(aqi))
            #print("in 2nd range aqi", df4['AQI'])
        elif drange(c,9.5,12.4)== 1:
            aqi = ((49/2.9)*(c-9.5)) + 101
            df = df.set_value(i, 'AQI' ,math.floor(aqi))
            #print("in 3rd range aqi", row['AQI'])
        elif drange(c,12.5,15.4)== 1:
            aqi = ((49/2.9)*(c-12.5)) + 151
            df = df.set_value(i, 'AQI' ,math.floor(aqi))
        elif drange(c,15.5,30.4)== 1:
            aqi = ((99/14.9)*(c-15.5)) + 201
            df = df.set_value(i, 'AQI' ,math.floor(aqi))
        elif drange(c,30.5,40.4)== 1:
            aqi = ((99/9.9)*(c-30.5)) + 301
            df = df.set_value(i, 'AQI' ,math.floor(aqi))
        elif drange(c,40.5,50.4)== 1:
            aqi = ((99/9.9)*(c-40.5)) + 401
            df = df.set_value(i, 'AQI' ,math.floor(aqi))
    print('Cleaned',count, "rows")
    return df

In [64]:
df = pd.read_csv('Summarized_daily_PM2.5.csv',low_memory=False)
print(df.isnull().sum())
df.drop(['Method_Code','CBSA_Name','Parameter_Code','Parameter_Name','Units_of_Measure','Pollutant_Standard','Sample_Duration','Event_Type','Local_Site_Name','Address','State_Name','County_Name','City_Name','CBSA_Name','Date_of_Last_Change','Method_Code','Method_Name'],axis=1,inplace =True)
df = merge_3_col(df)
df = cleanPM25(df)
rewrite_file(df, 'Summarized_daily_PM2.5.csv')
print(df.isnull().any().any())

State_Code                  0
County_Code                 0
Site_Num                    0
Parameter_Code              0
POC                         0
Latitude                    0
Longitude                   0
Datum                       0
Parameter_Name              0
Sample_Duration             0
Pollutant_Standard     706107
Date_Local                  0
Units_of_Measure            0
Event_Type                  0
Observation_Count           0
Observation_Percent         0
Arithmetic_Mean             0
Max_Value                   0
1st_Max_Hour                0
AQI                    706107
Method_Code            621836
Method_Name                 0
Local_Site_Name         76065
Address                    36
State_Name                  0
County_Name                 0
City_Name                   0
CBSA_Name              154714
Date_of_Last_Change         0
dtype: int64
Cleaned 706107 rows
False


In [63]:
def cleanPM25AQI(df):
    df['AQI'] = df['AQI'].fillna(5000) 
    count = 0
    for i,row in df.iterrows():
        if row['Max_Value'] <= 0:
            df = df.set_value(i, 'Max_Value' ,1)
        elif row['Max_Value'] >= 500.4:
            df = df.set_value(i, 'Max_Value' ,500.3)
        if row['AQI'] == 5000: 
            count =  count +1 
            c = row['Max_Value']
            #print(index, row['AQI'] , c)
            if  drange(c,0.0,12) == 1:
                #print("in 1st range", c)
                aqi = (50/12*c)
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
            elif drange(c,12.1,35.4) == 1:
                #print("in 2nd range", c)
                aqi = ((49/23.3)*(c-12.1)) + 51
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
                #print("in 2nd range aqi", df4['AQI'])
            elif drange(c,35.5,55.4)== 1:
                aqi = ((49/19.9)*(c-35.5)) + 101
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
                #print("in 3rd range aqi", row['AQI'])
            elif drange(c,55.5,150.4)== 1:
                aqi = ((49/94.9)*(c-55.5)) + 151
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
            elif drange(c,150.5,250.4)== 1:
                aqi = ((99/99.9)*(c-150.5)) + 201
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
            elif drange(c,250.5,350.4)== 1:
                aqi = ((99/99.9)*(c-250.5)) + 301
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
            elif drange(c,350.5,500.4)== 1:
                aqi = ((99/149.9)*(c-350.5)) + 401
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
    print('Cleaned',count, "rows")
    return df

In [53]:
df = pd.read_csv('Summarized_daily_PM10.csv',low_memory=False)
print(df.isnull().sum())
df.drop(['Method_Code','CBSA_Name','Datum','Parameter_Code','Parameter_Name','Units_of_Measure','Pollutant_Standard','Sample_Duration','Event_Type','Local_Site_Name','Address','State_Name','County_Name','City_Name','CBSA_Name','Date_of_Last_Change','Method_Code','Method_Name'],axis=1,inplace =True)
df = merge_3_col(df)
df = cleanPM10AQI(df)
rewrite_file(df, 'Summarized_daily_PM10.csv')
print(df.isnull().any().any())

State_Code             0
POC                    0
Latitude               0
Longitude              0
Date_Local             0
Observation_Count      0
Observation_Percent    0
Arithmetic_Mean        0
Max_Value              0
1st_Max_Hour           0
AQI                    0
dtype: int64
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 0.0
in drange 0.0
in drange 0.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.0
in drange 604.

PermissionError: [Errno 13] Permission denied: 'Summarized_daily_PM10.csv'

In [ ]:
def cleanPM10AQI(df):
    df['AQI'] = df['AQI'].fillna(5000) 
    count = 0 
    for i, row in df.iterrows():
        if row['Max_Value'] < 0:
            df = df.set_value(i, 'Max_Value' ,0)
        elif row['Max_Value'] > 604:
            df = df.set_value(i, 'Max_Value' ,604)
        if row['AQI'] == 5000:
            count =  count + 1
            c = row['Max_Value']
            if  drange(c,0,54.99) == 1:
                #print("in 1st range", c)
                aqi = (50/54)*c
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
            elif drange(c,55,154.99) == 1:
                #print("in 2nd range", c)
                aqi = ((49/99)*(c-55)) + 51
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
                #print("in 2nd range aqi", df4['AQI'])
            elif drange(c,155,254.99)== 1:
                aqi = ((49/99)*(c-155)) + 101
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
                #print("in 3rd range aqi", row['AQI'])
            elif drange(c,255,354.99)== 1:
                aqi = ((49/99)*(c-255)) + 151
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
            elif drange(c,355,424.99)== 1:
                aqi = ((99/69)*(c-355)) + 201
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
            elif drange(c,425,504.99)== 1:
                aqi = ((99/79)*(c-425)) + 301
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
            elif drange(c,505,604)== 1:
                aqi = ((99/99)*(c-505)) + 401
                df = df.set_value(i, 'AQI' ,math.floor(aqi))
    print('Cleaned',count, "rows")
    return df